In [ ]:

### vimp function
#' Variable importance for causal forests
#'
#' @param c.forest A causal forest from the grf package.
#' @param variable.groups List of group of variables for group variable importance. Default is NULL, and each input variable forms a group.
#'
#' @return The list of importance values for all input variables or provided variable groups.
#' @export
#'
#' @examples


vimp_causal_forests <- function(c.forest, variable.groups = NULL){

  # check input c.forest is a grf causal forest
  is.causal.forest <- all(class(c.forest) == c('causal_forest', 'grf'))
  if (!is.causal.forest){
    stop('c.forest must be a grf causal forest.')
  }

  # get initial data
  X <- c.forest$X.orig
  Y <- c.forest$Y.orig
  W <- c.forest$W.orig
  p <- ncol(X)
  n <- nrow(X)

  # get centered outcome and treatment assignment
  Y.centered <- Y - c.forest$Y.hat
  W.centered <- W - c.forest$W.hat

  # get oob predictions
  tau.hat <- c.forest$predictions

  # set variable groups
  if (is.null(variable.groups)){
    # when variable.groups is NULL, each input variable defines a group
    index.groups <- as.list(1:p)
  } else {
    # check provided variable groups are valid (non empty list, all variable names are contained in the data, and not all variables in one group)
    non.empty.list <- is.list(variable.groups) & length(variable.groups) > 0
    if (!non.empty.list){
      stop('variable.groups must be an non-empty list.')
    }
    names.valid <- all(unlist(variable.groups) %in% colnames(X))
    if (!names.valid){
      stop('Variable names provided in variable.groups not all found in input data.')
    }
    groups.valid <- max(sapply(variable.groups, function(group){length(unique(group))})) < p
    if (!groups.valid){
      stop('A group cannot contain all input variables.')
    }
    # get group of variable indices
    index.groups <- lapply(variable.groups, function(group){
      unique(sapply(group, function(j){which(colnames(X) == j)}))
    })
  }

  # compute vimp for all input variables using the settings of the initial causal forest
  In <- sapply(index.groups, function(index){
    c.forest.drop.Xj <- causal_forest(X[, -index, drop=F], Y, W, Y.hat = c.forest$Y.hat, W.hat = c.forest$W.hat,
                                      num.trees = c.forest$`_num_trees`,
                                      sample.weights = c.forest$sample.weights,
                                      clusters = c.forest$clusters,
                                      equalize.cluster.weights = c.forest$equalize.cluster.weights,
                                      sample.fraction = c.forest$tunable.params$sample.fraction,
                                      mtry = min(c.forest$tunable.params$mtry, ncol(X) - length(index)),
                                      min.node.size = c.forest$tunable.params$min.node.size,
                                      honesty.fraction = c.forest$tunable.params$honesty.fraction,
                                      honesty.prune.leaves = c.forest$tunable.params$honesty.prune.leaves,
                                      alpha = c.forest$tunable.params$alpha,
                                      imbalance.penalty = c.forest$tunable.params$imbalance.penalty,
                                      ci.group.size = c.forest$ci.group.size)
    alpha <- get_forest_weights(c.forest.drop.Xj)
    vimp.Xj <- compute_vimp(alpha, Y.centered, W.centered, tau.hat)
    return(vimp.Xj)
  })

  # compute retrain bias
  c.forest0 <- causal_forest(X, Y, W, Y.hat = c.forest$Y.hat, W.hat = c.forest$W.hat,
                                    num.trees = c.forest$`_num_trees`,
                                    sample.weights = c.forest$sample.weights,
                                    clusters = c.forest$clusters,
                                    equalize.cluster.weights = c.forest$equalize.cluster.weights,
                                    sample.fraction = c.forest$tunable.params$sample.fraction,
                                    mtry = c.forest$tunable.params$mtry,
                                    min.node.size = c.forest$tunable.params$min.node.size,
                                    honesty.fraction = c.forest$tunable.params$honesty.fraction,
                                    honesty.prune.leaves = c.forest$tunable.params$honesty.prune.leaves,
                                    alpha = c.forest$tunable.params$alpha,
                                    imbalance.penalty = c.forest$tunable.params$imbalance.penalty,
                                    ci.group.size = c.forest$ci.group.size)
  alpha <- get_forest_weights(c.forest0)
  In0 <- compute_vimp(alpha, Y.centered, W.centered, tau.hat)

  # compute debiased importance
  In <- In - In0

  return(In)

}

# compute vimp from coefficients alpha of the retrained forest, centered outcome and treatment assignment, and original oob predictions
compute_vimp <- function(alpha, Y.centered, W.centered, tau.hat){
  W.bar <- alpha%*%W.centered
  W2.bar <- alpha%*%(W.centered^2)
  Y.bar <- alpha%*%Y.centered
  tau.bar <- alpha%*%tau.hat
  tau.hat.new <- (alpha%*%(W.centered*Y.centered - tau.hat*(W.centered^2)) - (W.bar*Y.bar - tau.bar*W2.bar))/(W2.bar - W.bar^2)
  vimp <- as.numeric(sum((tau.hat - tau.hat.new)^2)/(length(Y.centered)*var(tau.hat)))
  return(vimp)
}


In [ ]:
install.packages("grf")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘zoo’, ‘DiceKriging’, ‘lmtest’, ‘Rcpp’, ‘sandwich’, ‘RcppEigen’




In [ ]:
install.packages("MASS")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [ ]:
# Check the version of the grf package
package_version <- as.character(packageVersion("grf"))

# Print the version information
cat("grf package version:", package_version, "\n")

# Load packages
library(grf)
library(MASS)

# Define data
p <- 8
n <- 3000
sample_data <- function(n, p){
  covmat <- diag(1, p)
  covmat[1,5] <- covmat[5,1] <- 0.9
  mu <- rep(0,p)
  X <- mvrnorm(n, mu = mu, Sigma = covmat)
  colnames(X) <- paste0('V', 1:p)
  prob <- 0.4 + 0.2 * (X[, 1] > 0)
  W <- rbinom(n, 1, prob)
  tau1 <- pmax(X[, 1], 0)
  tau2 <- 0.6*pmax(X[, 2], 0)
  tau <- tau2 + tau1
  mu <- X[, 3]^2*X[,4]^2
  Y <-  tau * W + mu + rnorm(n, sd = sqrt(0.1))
  return(list(X=X, W=W, Y=Y))
}

# Compute vimp with repetitions for uncertainties
nrep <- 10
vimpMC <- sapply(1:nrep, function(iter){

  # Print iteration number
  print(paste0('Computing iteration: ', iter, ' ...'))

  # Sample data
  data <- sample_data(n, p)

  # Fit initial forest
  tau_forest <- causal_forest(X = data$X, Y = data$Y, W = data$W)

  # Compute built-in variable importance using grf
  vimp <- variable_importance(tau_forest)

  return(vimp)

})

# Aggregate results
vimp_mean <- round(apply(vimpMC, 1, mean), 2)
vimp_sd <- round(apply(vimpMC, 1, sd)/sqrt(nrep), 2)
variables <- cbind(paste0('V', 1:p))
df_vimp <- data.frame(variables, vimp_mean, vimp_sd)
df_vimp <- df_vimp[order(-df_vimp[,2]),]

# Print results
print(df_vimp)



grf package version: 2.3.2 
[1] "Computing iteration: 1 ..."
[1] "Computing iteration: 2 ..."
[1] "Computing iteration: 3 ..."
[1] "Computing iteration: 4 ..."
[1] "Computing iteration: 5 ..."
[1] "Computing iteration: 6 ..."
[1] "Computing iteration: 7 ..."
[1] "Computing iteration: 8 ..."
[1] "Computing iteration: 9 ..."
[1] "Computing iteration: 10 ..."
  variables vimp_mean vimp_sd
1        V1      0.49    0.02
2        V2      0.13    0.02
4        V4      0.11    0.01
5        V5      0.11    0.01
3        V3      0.10    0.01
7        V7      0.02    0.00
8        V8      0.02    0.00
6        V6      0.01    0.00


In [ ]:
# Check the version of the grf package
package_version <- as.character(packageVersion("grf"))

# Print the version information
cat("grf package version:", package_version, "\n")

# Load packages
library(grf)
library(MASS)

# Define data
p <- 8
n <- 3000
sample_data <- function(n, p){
  covmat <- diag(1, p)
  covmat[1,5] <- covmat[5,1] <- 0.9
  mu <- rep(0,p)
  X <- mvrnorm(n, mu = mu, Sigma = covmat)
  colnames(X) <- paste0('V', 1:p)
  prob <- 0.4 + 0.2 * (X[, 1] > 0)
  W <- rbinom(n, 1, prob)
  tau1 <- pmax(X[, 1], 0)
  tau2 <- 0.6*pmax(X[, 2], 0)
  tau <- tau2 + tau1
  mu <- X[, 3]^2*X[,4]^2
  Y <-  tau * W + mu + rnorm(n, sd = sqrt(0.1))
  return(list(X=X, W=W, Y=Y))
}

# Compute vimp with repetitions for uncertainties
nrep <- 10
vimpMC <- sapply(1:nrep, function(iter){

  # Print iteration number
  print(paste0('Computing iteration: ', iter, ' ...'))

  # Sample data
  data <- sample_data(n, p)

  # Fit initial forest
  tau_forest <- causal_forest(X = data$X, Y = data$Y, W = data$W)

  # Compute built-in variable importance using grf
  vimp <- vimp_causal_forests(tau_forest)

  return(vimp)

})

# Aggregate results
vimp_mean <- round(apply(vimpMC, 1, mean), 2)
vimp_sd <- round(apply(vimpMC, 1, sd)/sqrt(nrep), 2)
variables <- cbind(paste0('V', 1:p))
df_vimp <- data.frame(variables, vimp_mean, vimp_sd)
df_vimp <- df_vimp[order(-df_vimp[,2]),]

# Print results
print(df_vimp)

grf package version: 2.3.2 
[1] "Computing iteration: 1 ..."
[1] "Computing iteration: 2 ..."
[1] "Computing iteration: 3 ..."
[1] "Computing iteration: 4 ..."
[1] "Computing iteration: 5 ..."
[1] "Computing iteration: 6 ..."
[1] "Computing iteration: 7 ..."
[1] "Computing iteration: 8 ..."
[1] "Computing iteration: 9 ..."
[1] "Computing iteration: 10 ..."
  variables vimp_mean vimp_sd
2        V2      0.23    0.02
1        V1      0.19    0.01
3        V3      0.04    0.01
4        V4      0.03    0.01
5        V5      0.00    0.00
6        V6      0.00    0.00
7        V7      0.00    0.00
8        V8      0.00    0.00


In [ ]:

package_version <- as.character(packageVersion("grf"))

# Print the version information
cat("grf package version:", package_version, "\n")

# Load packages
library(grf)
library(MASS)

# define data
p <- 8
n <- 3000
sample_data <- function(n, p){
  covmat <- diag(1, p)
  covmat[1,5] <- covmat[5,1] <- 0.9
  mu <- rep(0,p)
  X <- mvrnorm(n, mu = mu, Sigma = covmat)
  colnames(X) <- paste0('V', 1:p)
  prob <- 0.4 + 0.2 * (X[, 1] > 0)
  W <- rbinom(n, 1, prob)
  tau <- 0.6*pmax(X[, 2], 0)
  mu <- pmax(X[, 1], 0) + X[, 3]^2*X[,4]^2
  Y <-  tau * W + mu + rnorm(n, sd = sqrt(0.1))
  return(list(X=X, W=W, Y=Y))
}

# compute vimp with repetitions for uncertainties
nrep <- 10
vimpMC <- sapply(1:nrep, function(iter){

  # print iteration number
  print(paste0('Computing iteration: ', iter, ' ...'))

  ### sample data
  data <- sample_data(n, p)

  # fit initial forest
  tau_forest <- causal_forest(X = data$X, Y = data$Y, W = data$W)

  # compute vimp
  vimp <- vimp_causal_forests(tau_forest)

  return(vimp)

})

# aggregate results
vimp_mean <- round(apply(vimpMC, 1, mean), 3)
vimp_sd <- round(apply(vimpMC, 1, sd)/sqrt(nrep), 3)
variables <- cbind(paste0('X', 1:p))
df_vimp <- data.frame(variables, vimp_mean, vimp_sd)
df_vimp <- df_vimp[order(-df_vimp[,2]),]

# print results
print(df_vimp)


grf package version: 2.3.2 
[1] "Computing iteration: 1 ..."
[1] "Computing iteration: 2 ..."
[1] "Computing iteration: 3 ..."
[1] "Computing iteration: 4 ..."
[1] "Computing iteration: 5 ..."
[1] "Computing iteration: 6 ..."
[1] "Computing iteration: 7 ..."
[1] "Computing iteration: 8 ..."
[1] "Computing iteration: 9 ..."
[1] "Computing iteration: 10 ..."
  variables vimp_mean vimp_sd
2        X2     0.882   0.024
3        X3     0.172   0.032
4        X4     0.122   0.030
7        X7     0.008   0.002
6        X6     0.006   0.002
1        X1     0.004   0.001
8        X8     0.004   0.002
5        X5     0.003   0.001


In [ ]:

# load packages
#library(grf)
#library(MASS)

# vimp function
#source('vimp_causal_forests.R')

# define data
p <- 5
n <- 3000
sample_data <- function(n, p){
  X <- as.data.frame(matrix(runif(p*n), ncol = p))
  X[,1] <- X[,1]^3
  colnames(X) <- paste0('V', 1:p)
  prob <- X[,1]
  W <- rbinom(n, 1, prob)
  tau <- 10*prob*(1 - prob)
  Y <-  tau * W + X[,2] + rnorm(n, sd = sqrt(0.1))
  return(list(X=X, W=W, Y=Y, tau=tau))
}

# compute vimp with repetitions for uncertainties
nrep <- 10
vimpMC <- sapply(1:nrep, function(iter){

  # print iteration number
  print(paste0('Computing iteration: ', iter, ' ...'))

  ### sample data
  data <- sample_data(n, p)

  # fit initial forest
  tau_forest <- causal_forest(X = data$X, Y = data$Y, W = data$W)

  # compute vimp
  vimp <- vimp_causal_forests(tau_forest)

  return(vimp)

})

# aggregate results
vimp_mean <- round(apply(vimpMC, 1, mean), 4)
vimp_sd <- round(apply(vimpMC, 1, sd)/sqrt(nrep), 4)
variables <- cbind(paste0('X', 1:p))
df_vimp <- data.frame(variables, vimp_mean, vimp_sd)
df_vimp <- df_vimp[order(-df_vimp[,2]),]

# print results
print(df_vimp)


[1] "Computing iteration: 1 ..."
[1] "Computing iteration: 2 ..."
[1] "Computing iteration: 3 ..."
[1] "Computing iteration: 4 ..."
[1] "Computing iteration: 5 ..."
[1] "Computing iteration: 6 ..."
[1] "Computing iteration: 7 ..."
[1] "Computing iteration: 8 ..."
[1] "Computing iteration: 9 ..."
[1] "Computing iteration: 10 ..."
  variables vimp_mean vimp_sd
1        X1    0.9624  0.0041
2        X2    0.0004  0.0008
5        X5   -0.0002  0.0007
3        X3   -0.0003  0.0006
4        X4   -0.0010  0.0008
